<a href="https://colab.research.google.com/github/spriyam095/LeuronN/blob/master/EmotionRecognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#                 **SPEECH EMOTION RECOGNITON**

Speech emotion recognition is a simple Python mini-project, which we are going to practice. But before we go further we need to install a library(***soundfile***) to operate with our audio files.

In [8]:
pip install soundfile 

And because at the start of the Model I faced certain problems with sklearn_version 0.20+... . Therefore I got rid of it and went further with sklearn_version 0.19.1

In [9]:
pip uninstall sklearn

Uninstalling sklearn-0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/sklearn-0.0.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled sklearn-0.0


In [10]:
pip install scikit-learn==0.19.1

     |████████████████████████████████| 12.4MB 188kB/s 
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.1 which is incompatible.
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


Now we import necessary libraries. I discovered two new libraries in making this project. **Glob** and **Soundfile**.

In [1]:
import librosa
import soundfile
import glob , os , pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import sklearn
print(sklearn.__version__)

0.19.1


Now here I mount my google drive to access my dataset to train on.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Further below is the code to **extract features of those audios** whose emotions are mentioned in the dictionary further below in this notebook.

In [0]:
#Extract features (mfcc, chroma, mel) from a sound file

def extract_feature(file_name, mfcc, chroma, mel):

    with soundfile.SoundFile(file_name) as sound_file:

        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

Here we Initialize our **Dictionaries**. One may add any emotion from **Dict:** *emotions* to **Dict:** *observed_emotions*

In [0]:
#Emotions in the RAVDESS dataset

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

Here we define our func **load_data()** to load data in train & test respectively. The function below will call the func **extract_features()** to extract features of each audio in order to be studied to predict its emotion.

In [0]:
#Load the data and extract features for each sound file

def load_data(test_size=0.2):

    x,y=[],[]

    for file in glob.glob("/content/gdrive/My Drive/Colab Notebooks/Emotionaudio/Actor_*/*.wav"):
      
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]

        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)

    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

Next we call our func **load_data()**

In [0]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

**This is what our training data looks like**


![alt text](https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2019/09/dataset-simple-python-project.png)

![alt text](https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2019/09/dataset-2-interesting-python-projects.png)

Next we check the shapes of our train and test respectively.

In [15]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


Now we see number of features each audio has to be studied.

In [16]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


That been done next we initialize our training model

In [0]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=200, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=1300)

Next we train it on our training data.

In [70]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=200, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1300, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

Next we test our model on test_data and see the results.

In [73]:
#Predict for the test set
y_pred=model.predict(x_test)
print(y_pred)

['calm' 'disgust' 'happy' 'happy' 'disgust' 'calm' 'happy' 'happy'
 'disgust' 'happy' 'happy' 'disgust' 'happy' 'happy' 'calm' 'fearful'
 'calm' 'happy' 'disgust' 'fearful' 'calm' 'happy' 'disgust' 'disgust'
 'happy' 'happy' 'disgust' 'disgust' 'disgust' 'disgust' 'happy' 'disgust'
 'happy' 'disgust' 'fearful' 'disgust' 'calm' 'fearful' 'disgust'
 'disgust' 'happy' 'disgust' 'calm' 'calm' 'happy' 'happy' 'disgust'
 'happy' 'calm' 'fearful' 'happy' 'fearful' 'happy' 'happy' 'disgust'
 'calm' 'disgust' 'disgust' 'calm' 'calm' 'happy' 'calm' 'happy' 'fearful'
 'disgust' 'calm' 'calm' 'disgust' 'calm' 'calm' 'fearful' 'fearful'
 'happy' 'happy' 'fearful' 'disgust' 'happy' 'happy' 'disgust' 'fearful'
 'fearful' 'disgust' 'happy' 'calm' 'fearful' 'disgust' 'calm' 'disgust'
 'disgust' 'disgust' 'disgust' 'disgust' 'calm' 'fearful' 'happy' 'calm'
 'calm' 'disgust' 'calm' 'disgust' 'happy' 'disgust' 'disgust' 'calm'
 'happy' 'disgust' 'happy' 'happy' 'calm' 'fearful' 'fearful' 'happy'
 'calm' '

Now we check its accuracy on our test data.

In [72]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 63.54%


For now its 63.54%. But one can get even better results with few improvsations.

# THE END